# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import pathlib
from pathlib import Path
import random
import numpy as np
import tensorflow as tf
!pip install opencv-python
import cv2
from google.colab.patches import cv2_imshow

In [3]:
!pip install tensorflow-object-detection-api
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util

     |████████████████████████████████| 583kB 2.7MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 2.7MB 13.0MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-cp36-none-any.whl size=844515 sha256=15fb00c5ea77fd7f1306f4c54ac8ef7c243f38512cf72bcfb5e13c3635e180a1
  Stored in directory: /root/.cache/pip/wheels/4a/54/d0/cfca11930c4b2025d40dede77059094070a67cc3e7bd3b285f
Successfully built tensorflow-object-detection-api
ERROR: readme-renderer 25.0 has requirement Pygments>=2.5.1, but you'll have pygments 2.1.3 which is incompatible.


In [4]:
%%shell
#mkdir /content/drive/'My Drive'/Capstone1/computer_vision
#cd /content/drive/'My Drive'/Capstone1/computer_vision
git clone https://github.com/tensorflow/models.git

cd models/research
pwd
protoc object_detection/protos/*.proto --python_out=.

export PYTHONPATH=$PYTHONPATH:/content/drive/'My Drive'/Capstone1/computer_vision/models/research
export PYTHONPATH=$PYTHONPATH:/content/drive/'My Drive'/Capstone1/computer_vision/models/research/slim

python setup.py build
python setup.py install 

Cloning into 'models'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 34118 (delta 23), reused 36 (delta 5), pack-reused 34062
Receiving objects: 100% (34118/34118), 512.39 MiB | 33.05 MiB/s, done.
Resolving deltas: 100% (22001/22001), done.
Checking out files: 100% (2495/2495), done.
/content/models/research
running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/model_lib_v2.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> 

In [5]:
!unzip /content/drive/'My Drive'/NEU-DET.zip

Archive:  /content/drive/My Drive/NEU-DET.zip
   creating: NEU-DET/
   creating: NEU-DET/ANNOTATIONS/
  inflating: NEU-DET/ANNOTATIONS/crazing_1.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_10.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_100.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_101.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_102.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_103.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_104.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_105.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_106.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_107.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_108.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_109.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_11.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_110.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_111.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_112.xml  
  inflating: NEU-DET/ANNOTATIONS/crazing_113.xml  
  inflating: NEU-DET/ANNOTATIONS/cr

In [6]:
!unzip /content/drive/'Shared drives'/DAEN690/final_model.zip

Archive:  /content/drive/Shared drives/DAEN690/final_model.zip
   creating: final_model/saved_model/variables/
  inflating: final_model/model.ckpt.index  
  inflating: final_model/checkpoint  
  inflating: final_model/pipeline.config  
  inflating: final_model/model.ckpt.meta  
  inflating: final_model/saved_model/saved_model.pb  
  inflating: final_model/model.ckpt.data-00000-of-00001  
  inflating: final_model/frozen_inference_graph.pb  


In [0]:
!unzip /content/img

In [0]:
tf.gfile = tf.io.gfile

model_path = "/content/final_model/"
labels_path = "/content/steel_label_map.pbtxt"
image_dir = "/content/NEU-DET/IMAGES"
image_file_pattern = "*.jpg"
output_path = "/content/det-out"

PATH_TO_IMAGES_DIR = pathlib.Path(image_dir)
IMAGE_PATHS = sorted(list(PATH_TO_IMAGES_DIR.glob(image_file_pattern)))


category_index = label_map_util.create_category_index_from_labelmap(labels_path, use_display_name=True)
class_num = len(category_index)

In [0]:
def get_color_table(class_num, seed=255):
    random.seed(seed)
    color_table = {}
    for i in range(class_num):
        color_table[i] =[random.randint(0, 255) for _ in range(3)]
    return color_table

colortable = get_color_table(class_num)

In [0]:
def load_model(model_path):
    
    model_dir = Path(model_path) / "saved_model"
    #model_dir = "D:/GMU/Apress-Sam/TrainedModels/final_model-2/save_model"
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']
    return model

## Process Single Image through SSD Inference Engine

This function takes a single image paramater and processes with the trained SSD Inference Engine.

In [0]:
def run_inference_for_single_image(model, image):
    # parameters
    #     model: pointer to trained model (imference engine)
    #     image: pointer to a singe image for processing
    
    input_tensor = tf.convert_to_tensor(image)
    
    # since the model operates in batch mode we will use tf.newaxis
    input_tensor = input_tensor[tf.newaxis, ...]
    output_dict = model(input_tensor)
    
    # model is a tensor
    # output is a tensor
    # convert from tensor to numpy array
    # use index[0] to delete the batch dimension
    # using first num_detection
    num_detections = int(output_dict.pop('num_detections')) # note this is basic 'pop' from a stack
    output_dict = {key: value[0, :num_detections].numpy()
                   for key, value in output_dict.items()}
    output_dict['num_detections'] = num_detections
    
    # convert detection_clasesse to ints data type
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    
    # process model with masks
    if 'dection_masks' in output_dict:
        #adjust bounding box mask for image size
        detection_masks_reframe = utils_ops.reframe_box_masks_to_image_masks(
             output_dict['detection_masks'], output_dict['detection_boxes'],
             image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

## Function - draw bounding boxes

This function will draw bounding boxes around defects in steel images

In [0]:
def infer_object(model, image_path):
    #
    # using OpenCV to read and crate image in the context of numpy
    #
    # final output will be an image marked with boxes and labels
    #
    
    imagename = os.path.basename(image_path)
    print(imagename)
    image_np = cv2.imread(os.path.abspath(image_path))
    
    # detections
    
    output_dict = run_inference_for_single_image(model, image_np)
    
    # visualization
    
    for i in range(output_dict['detection_classes'].size):
        
        box = output_dict['detection_boxes'][i]
        classes = output_dict['detection_classes'][i]
        scores = output_dict['detection_scores'][i]
        
        if scores > 0.5:
            h = image_np.shape[0]
            w = image_np.shape[1]
            classname = category_index[classes]['name']
            classid = category_index[classes]['id']
            
            print(classname)
            print(classid)
            # draw bounding box
            
            cv2.rectangle(image_np, (int(box[1] * w), int(box[0] * h)), (int(box[3] * w), int(box[2] * h)), colortable[classid-1], 2)
            
            # label bounding box
            
            font = cv2.FONT_HERSHEY_COMPLEX_SMALL
            cv2.putText(image_np, str(classname) + ":" + str(scores),
                          (int(box[1] * w), int(box[0] * h)-5), font, 1, colortable[classid-1], 1, 1)
        else:
            break
        
        # persist results
        
        cv2.imwrite(output_path+"/"+imagename, image_np)
        cv2_imshow(image_np)
    

In [14]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [27]:
# retrive the model object

detection_model = load_model(model_path)

# run the inference engine

for image_path in IMAGE_PATHS:
    print(image_path)
    infer_object(detection_model, image_path)
cv2.waitKey(0)

Output hidden; open in https://colab.research.google.com to view.

In [0]:
def flip(image):
    """Flips an image vertically """
    return image[::-1] # remember that image is a numpy array

In [0]:
from moviepy.editor import VideoFileClip
write_output = '/content/output_video.mp4'
clip1 = VideoFileClip("/content/video/I0.mp4")
white_clip = clip1.fl_image(flip)
%time white_clip.write_videofile(write_output, audio=False)

# From tensorflow/models/
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

[MoviePy] >>>> Building video /content/output_video.mp4
[MoviePy] Writing video /content/output_video.mp4


100%|██████████| 12600/12600 [10:02<00:00, 20.91it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/output_video.mp4 

CPU times: user 7min 17s, sys: 25.6 s, total: 7min 43s
Wall time: 10min 3s


In [0]:
from moviepy.editor import VideoFileClip
write_output = '/content/output_video_2.mp4'
clip1 = VideoFileClip("/content/video/I2.mp4")
white_clip = clip1.fl_image(flip)
%time white_clip.write_videofile(write_output, audio=False)

# From tensorflow/models/
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

[MoviePy] >>>> Building video /content/output_video_2.mp4
[MoviePy] Writing video /content/output_video_2.mp4


100%|██████████| 1729/1729 [00:14<00:00, 122.61it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/output_video_2.mp4 

CPU times: user 3.33 s, sys: 243 ms, total: 3.58 s
Wall time: 14.3 s


In [0]:
def process_image(img):
    global param1
    global param2
    if param1 > param2:
        #do whatever
        param1 = param2 * 2
    return image

In [0]:
def detect(frame, net):
    height, width = frame.shape[:2]
    x = torch.permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))
    y = net(x)
    detections = y.data
    scale = torch.Tensor([width, height, width, height])
    # detections = [batch, number of classes, number of occurence, (score, x0, Y0, x1, y1)]
    for i in range(detections.size(1)):
        j = 0
        while detections[0, i, j, 0] >= 0.6:
            pt = (detections[0, i, j, 1:] * scale).numpy()
            cv2.rectangle(frame, (int(pt[0]), int(pt[1])), (int(pt[2]), int(pt[3])), (255, 0, 0), 2)
            cv2.putText(frame, labelmap[i - 1], (int(pt[0]), int(pt[1])), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            j += 1
    return frame

In [0]:
import imageio
import torch
from torch.autograd import Variable
detection_model = load_model(model_path)
# Detecting objects in video
for file in os.listdir(image_dir):
	print('Reading', file)
	video_reader = imageio.get_reader(os.path.join(image_dir, file))	# Reading video
	fps = video_reader.get_meta_data()['fps']	# gettinf fps of the image
	video_writer = imageio.get_writer(os.path.join("/content/", file), fps = fps)	# Writing back output image
	for i, frame in enumerate(video_reader):
		output_frame = detect(frame, detection_model)	# detecting objects frame by frame
		video_writer.append_data(output_frame)	# appending frame to vidoe
		print('frame ', i, 'done')
	video_writer.close()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore
Reading I0.mp4


UnboundLocalError: ignored